# Homework 2 (Due Thursday Dec 1, 6:29pm PST)

Please submit as a notebook in the format `HW2_FIRSTNAME_LASTNAME_USCID.ipynb` in a group chat to me and the TAs.

Your `USCID` is your student 10-digit ID.

### Part I.  Topic Modelling and Analysis (5pts)

Pick from **one** of the dataset options below:
* **Negative McDonalds Yelp reviews**: `datasets/mcdonalds-yelp-negative-reviews.csv`
* **[Top 5000 Udemy courses](https://www.kaggle.com/datasets/90eededa5561eee7f62c0e68ecdad14c2bdb58bc923834067025dee655a6083e?resource=download)** - a Kaggle dataset of the course descriptions of the top 5000 Udemy courses in 2022: `datasets/top5000_udemy.csv`

In your notebook, explore the data and perform topic modelling. You may use any vectorization or text preprocessing techniques we have discussed.

In order to earn full credit, you must:

* Show the **# of topics you tried, and explain why you ultimately decided on the final #**.
* Demonstrate **adequate text preprocessing (there are likely obvious stopwords / fuzzy matching / regex groupings that can be done to improve the final results)** - show what you tried.
* In 2-3 sentences: A **business analysis of these topics - what do they reveal as actionable next steps or insights for McDonalds or Udemy?** Please be specific in your recommendations/insights.
    - **Not specific**: *We recommend Amazon look into the quality of their toys, since the reviews show disatisfaction with the value of their product.*
    - **Specific**: *Amazon should explore more durable batteries/hardwares. For example, X% of reviews mention that the toys' batteries were broken or immediately died. This is part of a larger theme of components not being ready to use out the box, which often leads to disappointment on holiday occasions when children open up their gifts. See the following document snippets as examples:...*

#### 1. Loading Data 

In [112]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
reviews = pd.read_csv("../datasets/mcdonalds-yelp-negative-reviews.csv", encoding='latin-1')
text = reviews["review"].values

In [174]:
city_list = list(reviews['city'].unique())

In [113]:
reviews.head()

,_unit_id,city,review
0,679455653,Atlanta,"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care."
1,679455654,Atlanta,"Terrible customer service. I came in at 9:30pm and stood in front of the register and no one bothered to say anything or help me for 5 minutes. There was no one else waiting for their food inside either, just outside at the window. I left and went to Chickfila next door and was greeted before I was all the way inside. This McDonalds is also dirty, the floor was covered with dropped food. Obviously filled with surly and unhappy workers."
2,679455655,Atlanta,"First they ""lost"" my order, actually they gave it to someone one else than took 20 minutes to figure out why I was still waiting for my order.They after I was asked what I needed I replied, ""my order"".They asked for my ticket and the asst mgr looked at the ticket then incompletely filled it.I had to ask her to check to see if she filled it correctly.She acted as if she couldn't be bothered with that so I asked her again.She begrudgingly checked to she did in fact miss something on the ticket.So after 22 minutes I finally had my breakfast biscuit platter.As I left an woman approached and identified herself as the manager, she was dressed as if she had just awoken in an old t-shirt and sweat pants.She said she had heard what happened and said she'd take care of it.Well why didn't she intervene when she saw I was growing annoyed with the incompetence?"
3,679455656,Atlanta,I see I'm not the only one giving 1 star. Only because there is not a -25 Star!!! That's all I need to say!
4,679455657,Atlanta,"Well, it's McDonald's, so you know what the food is. This review reflects solely on the poor service. I have been to this location countless times over the years. They consistently fail on the service end of things. The order takers tend to be rude, no smiles, and a lot of ""sighs"" and ""lip smacking"" when you talk to them. So why go back you ask? This store benefits from being the only place to eat in this area. The next stop is at least 12 minutes away on the other side of town. Also I strongly believe in 2nd chances and know that not every business can satisfy everyone 100% of the time. I have given them many chances at earning a positive review. I could not recommend this location any less. If you can wait, take a pass. There are better McDonald's stores in Griffin, GA."


### 2. Data Cleaning and Text Preprocessing

In [114]:
reviews['reviews_processed'] = reviews['review']

# Remove punctuation
from textacy.preprocessing.remove import punctuation
reviews['reviews_processed'] = reviews['reviews_processed'].apply(punctuation)

# Convert to lowercase 
reviews['reviews_processed'] = reviews['reviews_processed'].map(lambda x: x.lower())

# Replace common entities/concepts 
from textacy.preprocessing.replace import urls, hashtags, numbers, emails, emojis, currency_symbols
reviews['reviews_processed'] = reviews['reviews_processed'].\
 apply(urls).\
 apply(hashtags).\
 apply(currency_symbols).\
 apply(emojis).\
 apply(emails)
# apply(numbers)
# we won't remove numbers because it gives us valuable information 

# Remove or normalize undesired text elements 
#from collections import Counter
#from textacy.preprocessing.normalize import quotation_marks, bullet_points
#quotes = ['"','“','”']

reviews.head(10)


,_unit_id,city,review,reviews_processed
0,679455653,Atlanta,"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care.",i m not a huge mcds lover but i ve been to better ones this is by far the worst one i ve ever been too it s filthy inside and if you get drive through they completely screw up your order every time the staff is terribly unfriendly and nobody seems to care
1,679455654,Atlanta,"Terrible customer service. I came in at 9:30pm and stood in front of the register and no one bothered to say anything or help me for 5 minutes. There was no one else waiting for their food inside either, just outside at the window. I left and went to Chickfila next door and was greeted before I was all the way inside. This McDonalds is also dirty, the floor was covered with dropped food. Obviously filled with surly and unhappy workers.",terrible customer service i came in at 9 30pm and stood in front of the register and no one bothered to say anything or help me for 5 minutes there was no one else waiting for their food inside either just outside at the window i left and went to chickfila next door and was greeted before i was all the way inside this mcdonalds is also dirty the floor was covered with dropped food obviously filled with surly and unhappy workers
2,679455655,Atlanta,"First they ""lost"" my order, actually they gave it to someone one else than took 20 minutes to figure out why I was still waiting for my order.They after I was asked what I needed I replied, ""my order"".They asked for my ticket and the asst mgr looked at the ticket then incompletely filled it.I had to ask her to check to see if she filled it correctly.She acted as if she couldn't be bothered with that so I asked her again.She begrudgingly checked to she did in fact miss something on the ticket.So after 22 minutes I finally had my breakfast biscuit platter.As I left an woman approached and identified herself as the manager, she was dressed as if she had just awoken in an old t-shirt and sweat pants.She said she had heard what happened and said she'd take care of it.Well why didn't she intervene when she saw I was growing annoyed with the incompetence?",first they lost my order actually they gave it to someone one else than took 20 minutes to figure out why i was still waiting for my order they after i was asked what i needed i replied my order they asked for my ticket and the asst mgr looked at the ticket then incompletely filled it i had to ask her to check to see if she filled it correctly she acted as if she couldn t be bothered with that so i asked her again she begrudgingly checked to she did in fact miss something on the ticket so after 22 minutes i finally had my breakfast biscuit platter as i left an woman approached and identified herself as the manager she was dressed as if she had just awoken in an old t shirt and sweat pants she said she had heard what happened and said she d take care of it well why didn t she intervene when she saw i was growing annoyed with the incompetence
3,679455656,Atlanta,I see I'm not the only one giving 1 star. Only because there is not a -25 Star!!! That's all I need to say!,i see i m not the only one giving 1 star only because there is not a 25 star that s all i need to say
4,679455657,Atlanta,"Well, it's McDonald's, so you know what the food is. This review reflects solely on the poor service. I have been to this location countless times over the years. They consistently fail on the service end of things. The order takers tend to be rude, no smiles, and a lot of ""sighs"" and ""lip smacking"" when you talk to them. So why go back you ask? This store benefits from being the only place to eat in this area. The next stop is at least 12 minutes away on the other side of town. Also I strongly believe in 2nd chances and know 

In [116]:
# Lemmatization 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None


# Shoutout to this git repo for providing this code 
# https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

reviews['reviews_processed'] = reviews['reviews_processed'].apply(lemmatize_sentence)

In [117]:
pd.set_option('display.max_colwidth', None)

In [118]:
# Removing stopwords using gensim 
from gensim.parsing.preprocessing import remove_stopwords
reviews['reviews_processed'] = reviews['reviews_processed'].apply(remove_stopwords)
reviews[['review',"reviews_processed"]].head(10)


,review,reviews_processed
0,"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care.",m huge mcds lover ve good far bad ve s filthy inside drive completely screw order time staff terribly unfriendly care
1,"Terrible customer service. I came in at 9:30pm and stood in front of the register and no one bothered to say anything or help me for 5 minutes. There was no one else waiting for their food inside either, just outside at the window. I left and went to Chickfila next door and was greeted before I was all the way inside. This McDonalds is also dirty, the floor was covered with dropped food. Obviously filled with surly and unhappy workers.",terrible customer service come 9 30pm stand register bother help 5 minute wait food inside outside window leave chickfila door greet way inside mcdonalds dirty floor cover dropped food obviously surly unhappy worker
2,"First they ""lost"" my order, actually they gave it to someone one else than took 20 minutes to figure out why I was still waiting for my order.They after I was asked what I needed I replied, ""my order"".They asked for my ticket and the asst mgr looked at the ticket then incompletely filled it.I had to ask her to check to see if she filled it correctly.She acted as if she couldn't be bothered with that so I asked her again.She begrudgingly checked to she did in fact miss something on the ticket.So after 22 minutes I finally had my breakfast biscuit platter.As I left an woman approached and identified herself as the manager, she was dressed as if she had just awoken in an old t-shirt and sweat pants.She said she had heard what happened and said she'd take care of it.Well why didn't she intervene when she saw I was growing annoyed with the incompetence?",lose order actually 20 minute figure wait order ask need reply order ask ticket asst mgr look ticket incompletely ask check correctly act couldn t bother ask begrudgingly check fact miss ticket 22 minute finally breakfast biscuit platter leave woman approach identify manager dress awake old t shirt sweat pant hear happen d care t intervene saw grow annoy incompetence
3,I see I'm not the only one giving 1 star. Only because there is not a -25 Star!!! That's all I need to say!,m 1 star 25 star s need
4,"Well, it's McDonald's, so you know what the food is. This review reflects solely on the poor service. I have been to this location countless times over the years. They consistently fail on the service end of things. The order takers tend to be rude, no smiles, and a lot of ""sighs"" and ""lip smacking"" when you talk to them. So why go back you ask? This store benefits from being the only place to eat in this area. The next stop is at least 12 minutes away on the other side of town. Also I strongly believe in 2nd chances and know that not every business can satisfy everyone 100% of the time. I have given them many chances at earning a positive review. I could not recommend this location any less. If you can wait, take a pass. There are better McDonald's stores in Griffin, GA.",s mcdonald s know food review reflect solely poor service location countless time year consistently fail service end thing order taker tend rude smile lot sigh lip smacking talk ask store benefit place eat area stop 12 minute away town strongly believe 2nd chance know business satisfy 100 time chance earn positive review recommend location wait pas good mcdonald s store griffin ga
5,This has to be one of the worst and slowest McDonald's franchises there is. Can't figure out why my Egg McMuffin is always on a stale un-toasted English muffin. Bought A chocolate shake today and threw it away.,bad slow mcdonald s franchise t figure egg mcmuffin stale toast english muffin buy chocolate shake today throw away
6,"I'm not crazy about this McDonald's. This is primar

In [119]:
# Vectorize the corpus 
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(3,3), min_df=3,
                            max_df=0.4, stop_words="english")

X, terms = vectorizer.fit_transform(reviews['reviews_processed']), vectorizer.get_feature_names_out()
tf_idf = pd.DataFrame(X.toarray(), columns=terms)

print(f"TF-IDF: {tf_idf.shape}")
tf_idf.head(5)

TF-IDF: (1525, 223)


,10 minute food,10 minute fry,10 minute later,10 minute line,10 minute order,10 piece chicken,10 piece nugget,15 min food,15 minute drive,15 minute later,...,window hand drink,window pick food,wish negative star,work customer service,work drive order,work fast food,worst customer service,write review fast,write review mcdonald,write review mcdonalds
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
# Fit the NMF Model
nmf = NMF(n_components=3)
W = nmf.fit_transform(X)
H = nmf.components_
print(f"Original shape of X is {X.shape}")
print(f"Decomposed W matrix is {W.shape}")
print(f"Decomposed H matrix is {H.shape}")

Original shape of X is (1525, 223)
Decomposed W matrix is (1525, 3)
Decomposed H matrix is (3, 223)


/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [169]:
from typing import List
import numpy as np
# Report Results 
def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
      print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

get_top_tf_idf_tokens_for_topic(H, tf_idf.columns.tolist(), 3)

TOPIC 0

open 24 hour (39.8%)

drive open 24 (8.2%)

24 hour drive (4.8%)

TOPIC 1

wait 10 minute (41.0%)

order wrong time (7.0%)

10 minute order (5.8%)

TOPIC 2

fast food place (21.2%)

fast food restaurant (15.3%)

wait 15 minute (10.1%)



In [178]:
reviews[reviews['city']=='Dallas']

,_unit_id,city,review,reviews_processed
533,679456194,Dallas,"So it's fast food and McDonalds at that. So let's say expectations are very, very low. Sometimes though you just want food that's fast and you know what to expect. Not every meal can be a 5 star dinner with an executive chef.Even with appropriate low expectations, this location takes the cake for subpar service. About a week ago I was exhausted and needed something to eat and there are few fast food restaurants near where I live. So off to McDonalds I went. Ordered a value meal and a single apple pie. Surprise...no pie! Ok, mistakes happen. I've worked drive through in a previous life and I get it. Even though it was 10 pm and I was the only car there, so it's not like they were rushed. Still...mistakes happen.Fast forward to tonight. Again about 10 pm and exhausted and decide to go crazy and get McD's twice in 1 week. So I try again to order an apple pie with my value meal. Now, maybe this is my fault, because I should have checked the bag before driving off. But after sitting at the window and waiting for 5 minutes (and remember, I was really tired), silly me thought they'd be able to get it straight this time.Turns out I was wrong. Yet again...no pie. Both times (within a week of each other) I paid for them. Neither time they felt it necessary to put it in the actual bag.So I called to talk to the manager. And get hung up on as soon as I ask for the manager. So I called back, the manager actually answered and I told him what happened. He didn't sound too displeased, just told me ""next time we'll get your order right.""Yeah, if there IS a next time.",s fast food mcdonalds let s expectation low want food s fast know expect meal 5 star dinner executive chef appropriate low expectation location cake subpar service week ago exhaust need eat fast food restaurant near live mcdonalds ordered value meal single apple pie surprise pie ok mistake happen ve work drive previous life 10 pm car s like rush mistake happen fast forward tonight 10 pm exhaust decide crazy mcd s twice 1 week try order apple pie value meal maybe fault check bag drive sit window wait 5 minute remember tire silly think d able straight time turn wrong pie time week pay time felt necessary actual bag talk manager hang soon ask manager manager actually answer tell happen t sound displeased tell time ll order right yeah time
534,679456195,Dallas,"I know what your thinking....""another 1 star review for McD's and you don't even eat there?!? What gives?"" Well I have to say this is an ""informative"" review! Getting right down to it. I was out running by a few client sites today when I stopped and picked up lunch at Riverside (that's right, be jealous). So, a coworker calls and asks if I would pick up lunch for her. Being a team player I said no problem. She then told me she wanted Satan...I mean McDonalds. I sighed, but agreed. She told me what she wanted and off I went. Oh and to get some ketchup and Sweet and Sour sauce. Mkay. I pulled up to the drive thru and ordered after they tried to sell me their new Angus Burger. (Sidenote: Why do they do that in the first place? I mean, if that is what I wanted, I would have ordered it.) ""May I please have #12 with a Sprite."" ""Would you l like to add on an apple pie for $.50?"" ""Um, no."" ""$7.01, please pull up to the first window."" So I pay. Drive to window #2. Man opens the window hands me the drink and the bag of food. I asked for a S&S Sauce packet. The guy responded. ""We charge for that now."" ""Ha, that's hilarious. Just one please."" ""No really, they are $.28"" ""What? You're kidding right?"" ""No ma'am, if you want one you'll have to pull back around the drive thru."" (Totally dumbfounded I say) ""Do you charge for ketchup too?"" Then drive off. I laugh the entire way back to work like a crazy woman...""What do you mean your charging for condiments? "" ""Is it laced with heroin?"" ""Are there diamonds inside these packets?"" ""Is charging for this e

In [180]:
vectorizer = TfidfVectorizer(ngram_range=(2,2), min_df=3,
                            max_df=0.4, stop_words="english")

for city in city_list:
    print("================" + city + "================")
    reviews_city = reviews[reviews['city']==city]
    X, terms = vectorizer.fit_transform(reviews_city['reviews_processed']), vectorizer.get_feature_names_out()
    tf_idf = pd.DataFrame(X.toarray(), columns=terms)
    nmf = NMF(n_components=3)
    W = nmf.fit_transform(X)
    H = nmf.components_
    get_top_tf_idf_tokens_for_topic(H, tf_idf.columns.tolist(), 3)

================Atlanta================
TOPIC 0

fast food (51.5%)

particular location (4.4%)

expect mcdonald (3.1%)

TOPIC 1

customer service (22.7%)

place order (8.9%)

order drive (7.9%)

TOPIC 2

order wrong (23.1%)

northside hospital (21.8%)

ice cream (11.4%)

================Las Vegas================


/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


TOPIC 0

order right (4.0%)

customer service (3.0%)

fast food (3.0%)

TOPIC 1

big mac (21.5%)

look like (6.8%)

order big (4.9%)

TOPIC 2

ice cream (17.7%)

cream machine (9.6%)

sweet tea (6.2%)

================Dallas================
TOPIC 0

customer service (60.1%)

play area (12.7%)

place order (6.6%)

TOPIC 1

parking lot (74.1%)

look like (9.9%)

tell want (8.8%)

TOPIC 2

fast food (28.9%)

15 minute (11.5%)

look like (8.4%)

================Portland================
TOPIC 0

fast food (64.2%)

window order (6.5%)

food place (3.9%)

TOPIC 1

24 hour (76.2%)

big mac (14.1%)

food hot (5.3%)

TOPIC 2

time order (23.5%)

chicken sandwich (15.5%)

big mac (12.9%)

================Chicago================
TOPIC 0

parking lot (43.3%)

regular mcdonald (5.9%)

read review (4.5%)

TOPIC 1

customer service (32.2%)

dollar menu (6.0%)

great customer (5.7%)

TOPIC 2

look like (4.4%)

french fry (4.0%)

time order (3.6%)

================Cle

/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/jewonju/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and

TOPIC 0

customer service (30.2%)

bad customer (6.7%)

long line (6.0%)

TOPIC 1

mcdonald ve (11.0%)

look like (8.1%)

big mac (8.0%)

TOPIC 2

bad mcdonalds (43.9%)

staff like (8.3%)

20 minute (6.3%)



TypeError: can only concatenate str (not "float") to str

In [122]:
import numpy as np
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
            print("=" * 50)

In [123]:
get_top_documents_for_each_topic(W, reviews['review'].tolist())

Topic 0
100.0% Open 24 hours, right around the corner from my hotel. Let me just say, the dude working the graveyard shift is NOT very friendly. I don't remember his full name but it started with a C. You can just tell he was about one more "Would you like it Super Sized," away from completely going postal. I had the pleasure of running into him twice during my entire stay (what can i say? i like Mcdonald's.). Our second encounter was the weirdest. He started like mouthing something weird to me. I was too drunk to make it out, but I'm sure it was something along the lines as, "I'm going to shove these Chicken Nuggets up your ass."
100.0% Denied me a vanilla cone b/c it was "too late" assholes. Whats the point in being open for 24 hours?
100.0% pretty simple review, i work the early morning shift and they are open 24 hours. I've gone twice in the past week to get a coffee and one time they had to pull out the pot and start making it and the other time they told me they only were accepti

In [124]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3)
W = lda.fit_transform(X)
H = lda.components_
get_top_tf_idf_tokens_for_topic(H, tf_idf.columns.tolist(), 5)

TOPIC 0

open 24 hour (4.5%)

wait 15 minute (2.8%)

eat fast food (2.6%)

bad mcdonald ve (2.3%)

egg cheese biscuit (2.2%)

TOPIC 1

wait 10 minute (4.4%)

order wrong time (2.8%)

ice cream cone (2.4%)

free wi fi (2.3%)

time order wrong (1.8%)

TOPIC 2

fast food restaurant (3.1%)

fast food place (3.0%)

bad customer service (2.8%)

24 hour drive (2.3%)

ice cream machine (2.1%)



In [125]:
get_top_documents_for_each_topic(W, reviews['review'].tolist())

Topic 0
80.3% Normally I don't review a chain unless something about that particular location stands out, good or bad.This location for some reason, has the most ridiculous service I've ever encountered. It's McDonalds you say, what do you expect? Not much, really but let me tell you...ENCOUNTER #1I ordered 10-piece chicken nuggets, fries, and unsweetened ice tea. I got the nuggets and tea. I went up to the counter, placed my receipt on the counter gently faced away from me, and said "I'm sorry, I --" Before I finished my sentence, the employee SLAMMED her hand on my receipt on the counter and dragged it towards her. Um... WTF?!I didn't know how to react because she looked at my receipt and threw it away without addressing me. I went "um...ok..." and backed away from the counter to fill my tea. I have no clue what I did to piss this employee off but my friend who came with me, asked me what the heck happened. I said "I have no clue, I'm just missing my fries..." My friend was nice and 

### Part II. Emotion Classification (5 pts)

Use the `datasets/emotions_dataset.zip` (see the original Dataset source on [Kaggle](https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp)) to build a classification model that predicts the emotion of sentence. If you would like, you may classify only the top 4 emotions, and group all other classes as `Other`. 

In order to earn full credit, you must:

* Show the performance of your model with `CountVectorizer`, `TfIdfVectorizer`, `word2vec`, and `glove` embeddings.
    - for `word2vec`, make sure not to use the `en_core_web_sm` dataset (these are not real embeddings)
* Perform text preprocessing (or explain why it was not necessary):
    - stopword removal
    - ngram tokenization
    - stemming/lemmatization
    - fuzzy matching / regex cleaning / etc. (as you deem necessary, but show that you analyzed the text to make your decision)
* Show **AUROC / F1 scores** for on the holdout (test + validation) datasets.
* A brief discussion (2-3 sentences) of what could improve your model and why.

### 1. Importing Datasets 

In [148]:
train_df = pd.read_csv('../datasets/emotions/train.txt',header=None, names=['text'])
train_df[['text','emotion']] = train_df['text'].str.split(';',expand=True)
train_df['type'] = "train"
test_df = pd.read_csv('../datasets/emotions/test.txt',header=None, names=['text'])
test_df[['text','emotion']] = test_df['text'].str.split(';',expand=True)
test_df['type'] = "test"
val_df = pd.read_csv('../datasets/emotions/val.txt',header=None, names=['text'])
val_df[['text','emotion']] = val_df['text'].str.split(';',expand=True)
val_df['type'] = "val"

df = train_df.append(test_df.append(val_df , ignore_index=True) , ignore_index=True)

df.head()

/var/folders/zz/gt_tn3454131nr1dxcqk06b40000gn/T/ipykernel_12070/1303294747.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train_df.append(test_df.append(val_df , ignore_index=True) , ignore_index=True)


,text,emotion,type
0,i didnt feel humiliated,sadness,train
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,train
2,im grabbing a minute to post i feel greedy wrong,anger,train
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,train
4,i am feeling grouchy,anger,train


In [149]:
# Checking if all the data have been appended correctly
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)
print(df.shape)

print(df['type'].unique())

(16000, 3)
(2000, 3)
(2000, 3)
(20000, 3)
['train' 'test' 'val']


### 2. Text Preprocessing


In [150]:
# Removing stopwords using gensim 
from gensim.parsing.preprocessing import remove_stopwords
df['text_clean'] = df['text'].apply(remove_stopwords)
df.head(10)

,text,emotion,type,text_clean
0,i didnt feel humiliated,sadness,train,didnt feel humiliated
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,train,feeling hopeless damned hopeful cares awake
2,im grabbing a minute to post i feel greedy wrong,anger,train,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,train,feeling nostalgic fireplace know property
4,i am feeling grouchy,anger,train,feeling grouchy
5,ive been feeling a little burdened lately wasnt sure why that was,sadness,train,ive feeling little burdened lately wasnt sure
6,ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny,surprise,train,ive taking milligrams times recommended ive fallen asleep lot faster feel like funny
7,i feel as confused about life as a teenager or as jaded as a year old man,fear,train,feel confused life teenager jaded year old man
8,i have been with petronas for years i feel that petronas has performed well and made a huge profit,joy,train,petronas years feel petronas performed huge profit
9,i feel romantic too,love,train,feel romantic
